## Mancala rules

* Players sit on opposite sides of the long edge of the board
* There are 6 small pits in the middle of the board and 2 large ones at each end.  The small ones in the middle and the large pit on your right are yours.  The small ones on the other side and the large pit to your opponent's right are theirs
* The large pits at the end of the board are called Mancalas
* Set up the board with 4 stones per small pit (none in the mancalas)
* On every turn, select a pit on your side of the board that contains one or more stones,  then distribute its stones, one stone per pit, in an counter-clockwise direction until you have no stones remaining
* If you encounter your opponent's mandala, skip it
* If you encounter your mancala, drop a stone into it
* If the last stone lands in an empty pit on your side of the board, capture this stone and any stones in your opponent's pit on the other side of the board, collect all of these stones, including the one that just landed, and place them into your mancala.
* If either player's pits are entirely empty, the game concludes. 
* The player who still has stones on his side of the board when the game concludes places all of these pieces into their mancala.
The player with the most stones in their mancala is declared the winner. If both players have an equal number of stones in their mancala, the game results in a tie.


In [3]:
import random
random.seed(109)

In [6]:
class Mancala(Game):
    def __init__(self, pits_per_player = 6, stones_per_pit = 4):
        """
        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        initial_state = (board, 1)
        super().__init__(initial=states)
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0
        
        # Needed for intermediate report
        self.p1_turn_amount = 0
        self.p2_turn_amount = 0

    def actions(self, state):
        # Return a list of legal moves for the current player
        board, player = state
        actions = []
        for pit in range(1, self.pits_per_player + 1): 
            if(self.valid_move(pit)):
                actions.append(pit)

    def result(self, state, move):
        # Return a new state after applying the move
        board, player = state
        new_board = board[:]
        self.play(move)
        next_player = 3 - self.current_player
        return (new_board, next_player)

    def utility(self, state, player):
        # Return utility value (how good this state is for player)
        p1_score = board[self.p1_mancala_index]
        p2_score = board[self.p2_mancala_index]
        return (p1_score - p2_score) if player == 1 else (p2_score - p1_score)
    
    def terminal_test(self, state):
        # Return True if the game is over
        if winning_eval != 0:
            return True
        else: 
            return False
        
    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        if pit < 1 or pit > self.pits_per_player:
            return False
        if self.current_player == 1:
            if self.board[pit-1] == 0:
                return False
        elif self.current_player == 2:
            if self.board[pit+self.pits_per_player] == 0:
                return False
        return True
            
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        
        while True:
            pit = random.randint(1, self.pits_per_player)
            if self.current_player == 1:   
                if self.board[pit - 1] > 0:
                    return pit
            else:
                if self.board[pit + self.pits_per_player] > 0:
                    return pit
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        #print("Player " + str(self.current_player) + " choose pit: " + str(pit))
        
       
        # Check for valid move
        if not self.valid_move(pit): 
            print("INVALID MOVE")
            return self.board
        
        # Keep track of turns for intermediate report
        if self.current_player == 1:
            self.p1_turn_amount += 1
        else: self.p2_turn_amount += 1
        
        '''
           do i want to check for a winning state here as well
        '''
        
        # Append to moves list
        self.moves.append((self.current_player, pit))

        # Set values for helper function
        if self.current_player == 1:
            current_mancala = self.p1_mancala_index
            opponent_mancala = self.p2_mancala_index
            
            start_pit = pit-1 # Account for 0-based index
        else:
            current_mancala = self.p2_mancala_index
            opponent_mancala = self.p1_mancala_index
            
            start_pit = pit + self.pits_per_player # Account for 0-based index and mancala index
            
        # Collect stones and zero the choosen pit  
        stoneCount = self.board[start_pit]
        self.board[start_pit] = 0
        
        # Find first pit to be dropped and distribute stones
        currentPit = start_pit + 1
        self.play_helper(stoneCount, currentPit, opponent_mancala, current_mancala)
        
        # Check for winning state
        win = self.winning_eval() # Will return 1 (player 1 wins), 2 (player 2 wins), 3 (tie) or 0 (no winner yet)
        if win:
            win_messages = {
                1: "\nGAME OVER. Player 1 wins",
                2: "\nGAME OVER. Player 2 wins",
                3: "\nGAME OVER. Tie"
            }
            print(win_messages.get(win, "Unexpected Value"))
            print("Player 1 Mancala Value: " + str(self.board[self.p1_mancala_index]))
            print("Player 2 Mancala Value: " + str(self.board[self.p2_mancala_index]))
            return win
            
        # Change player
        self.current_player = 2 if self.current_player == 1 else 1
        
        return self.board
    
    def play_helper(self, stoneCount, currentPit, opponent_mancala, current_mancala):
        lower, upper = (self.p1_pits_index if self.current_player == 1 else self.p2_pits_index)
        while stoneCount:
            #print("currentPit: " + str(currentPit))
            
            # Don't drop stone in opponent's mancala
            if currentPit == opponent_mancala: 
                currentPit = (currentPit + 1) % len(self.board)
                continue
                
            # Drop a stone
            self.board[currentPit] += 1
            stoneCount -= 1 
                
            # Check for capture of stones
            if stoneCount == 0 and self.board[currentPit] == 1 and currentPit in range(lower, upper+1):
                across_index = self.find_across(currentPit) # Find opposite index from current pit
                
                # Check if opposite index has stones
                if self.board[across_index] > 0:
                    self.board[current_mancala] += 1 + self.board[across_index] # Add 1 stone plus opponents stones
                    self.board[across_index] = 0 # Set opponents stones to 0
                    self.board[currentPit] = 0 # Set current pit stones to 0
                    return
                
            # Move to next pit
            currentPit = (currentPit + 1) % len(self.board)
            
                 
    def find_across(self, index):
        """
        This function finds the index directly across from the given index
        """
        if self.current_player == 1:
            return self.p2_pits_index[1] - (index - self.p1_pits_index[0])
        if self.current_player == 2:
            return self.p1_pits_index[1] - (index - self.p2_pits_index[0])
        
        return None            
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        If either of the players' pits are all empty, then it is considered a winning state.
        
        Function will return 1 (player 1 wins), 2 (player 2 wins), 3 (tie), or 0 (no winner yet)
        """
        
        player1board = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player2board = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        
        # Check for either players' pits being all empty
        if all(x == 0 for x in player1board) or all(x == 0 for x in player2board):
            # Move all of player 1's stones into mancala
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1]+1):
                self.board[self.p1_mancala_index] += self.board[i]
                self.board[i] = 0
            # Move all of player 2's stones into mancala
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1]+1):
                self.board[self.p2_mancala_index] += self.board[i]
                self.board[i] = 0
            
            # Determine the winner
            if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                return 1 # Player 1 wins
            elif self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]:
                return 2 # Player 2 wins
            else: return 3 # Tie
 
        return 0
    

SyntaxError: invalid character '−' (U+2212) (398276110.py, line 53)

In [5]:
wins = {  # Initalize dictionary for keeping track of wins
    1: 0, # Player 1
    2: 0, # Player 2
    3: 0  # Tie
}

games_played = 100

p1_turn_amount, p2_turn_amount = 0, 0

for i in range(games_played):
    print("\nGAME NUMBER: " + str(i+1))
    game = Mancala(6,4)
    #game.display_board()
    while True:
        val = game.play(game.random_move_generator()) # Will return the board if no winner, or 1 (player 1 wins), 2 (player 2 wins), 3 (tie)
        if isinstance(val, int): # Check if a int is being returned
            wins[val] += 1
            p1_turn_amount += game.p1_turn_amount
            p2_turn_amount += game.p2_turn_amount
            
            
            '''
            print("\nList of valid moves:") # Printing the list of moves
            for move in game.moves:
                player, pit = move
                print(f"Player {player} selected pit {pit}")
            '''    
            break
        #game.display_board()
        
print("\nGame Statistics: Player 1")
print(f"Games Won: {wins[1] / games_played * 100:.2f}%")
print(f"Games Lost: {wins[2] / games_played * 100:.2f}%")
print(f"Games Tied: {wins[3] / games_played * 100:.2f}%")
print(f"Average Number of Turns over {games_played} games played: {p1_turn_amount/games_played}")

print("\nGame Statistics: Player 2")
print(f"Games Won: {wins[2] / games_played * 100:.2f}%")
print(f"Games Lost: {wins[1] / games_played * 100:.2f}%")
print(f"Games Tied: {wins[3] / games_played * 100:.2f}%")
print(f"Average Number of Turns over {games_played} games played: {p2_turn_amount/games_played}")


GAME NUMBER: 1


NameError: name 'Mancala' is not defined